# SSMR: generate SSM model from trajectory data

In [1]:
import numpy as np
from copy import deepcopy
from os.path import join

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

In [3]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [4]:
np.set_printoptions(linewidth=300)

## Settings for SSM model

In [5]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

SSMOrder = 3
obsDim = 2*oDOF
ROMOrder = 3
RDType = 'flow'

## Import and inspect decay data

In [6]:
if observables == "pos-vel":
    output_node = 'all'
elif observables == "delay-embedding":
    output_node = TIP_NODE
t_in = 1
t_out = 4

robot_dir = "../../../soft-robot-control/examples/trunk"
data_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/origin"
rest_file = join(robot_dir, 'rest_qv.pkl')


decayData_dir =  join(data_dir, "decay") # join(robot_dir, "dataCollection/origin") # 
Data = {}
Data['oData'] = utils.import_pos_data(decayData_dir, rest_file=rest_file, output_node=output_node, t_in=t_in, t_out=t_out)

nTRAJ = len(Data['oData'])
print("nTRAJ:", nTRAJ)

KeyboardInterrupt: 

Mainly required for adiabatic setting: Load info.yaml to get pre-tensioning that was used to obtain the decay data as well as the resulting equilibrium position

In [ ]:
import yaml
with open(join(decayData_dir, "info.yaml"), "r") as f:
    info = yaml.safe_load(f)
# if 'qv_eq' in info:
#     qv_eq = np.array(info['qv_eq'])
#     print(f"Loaded offset equilibrium position from info.yaml: {qv_eq}")
# else:
q_eq = np.mean([Data['oData'][i][1][:, -1] for i in range(nTRAJ)], axis=0)
u_eq = np.array(info['pre_tensioning'])
# qv_eq = np.hstack((q_eq, np.zeros_like(q_eq)))
# info['q_eq'] = q_eq.astype(float).tolist()
# print(f"Computed equilbrium position from data and saved to info.yaml: {q_eq}")
# with open(join(decayData_dir, "info.yaml"), "w") as f:
#     yaml.dump(info, f, default_flow_style=True)
print(u_eq)
print(q_eq)

Change of coordinates: shift oData to the offset equilibrium position

In [ ]:
# if observables == "delay-embedding":
#     rest_position = q_eq[:3]
# elif observables == "pos-vel":
#     rest_position = q_eq[]
for i in range(nTRAJ):
    Data['oData'][i][1] = (Data['oData'][i][1].T - q_eq).T

## Observables

EITHER compute reduced coordinates using delay embedding on the available oData OR use position and velocity of all nodes as observables

In [ ]:
if observables == "delay-embedding":
    # observables are position of tip + n_delay delay embeddings of the tip position
    assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)
elif observables == "pos-vel":
    # observables is position and velocity of tip node
    def assemble_observables(oData):
        if oData.shape[0] > 6:
            tip_node_slice = np.s_[3*TIP_NODE:3*TIP_NODE+3]
        else:
            tip_node_slice = np.s_[:3]
        return np.vstack((oData[tip_node_slice, :], np.gradient(oData[tip_node_slice, :], DT, axis=1)))

Compute observables

In [ ]:
Data['yData'] = deepcopy(Data['oData'])
for i in range(nTRAJ):
    Data['yData'][i][1] = assemble_observables(Data['oData'][i][1])

### Plot observables of interest

In [ ]:
outdofs = [0, 1, 2]
plt.close('all')
# plot trajectories in 3D [x, y, z] space
plot.traj_3D(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'])
# plot evolution of x, y and z in time, separately in 3 subplots
highlight_traj = [] # [14, 16, 18, 31, 35, 44] # []
plot.traj_xyz(Data,
             xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[1]), ('yData', outdofs[2])],
             xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
             highlight_idx=highlight_traj)
if observables == "pos-vel":
    # plot trajectories in 3D [x, x_dot, z] space / NB: x_dot = v_x
    plot.traj_3D(Data,
                xyz_idx=[('yData', outdofs[0]), ('yData', outdofs[0]+oDOF), ('yData', outdofs[2])],
                xyz_names=[r'$x$ [mm]', r'$\dot{x}$ [mm/s]', r'$z$ [mm]'])

Slice trajectories, i.e. remove time before they converge with SSM and cut off the end after complete decay

In [ ]:
t_interval = [0.1, 3]
Data['oDataTrunc'] = utils.slice_trajectories(Data['oData'], t_interval)
Data['yDataTrunc'] = utils.slice_trajectories(Data['yData'], t_interval)

## Obtain reduced-order coordinates

### Perform PCA on delay-embedded coordinates to obtain reduced coordinates

In [ ]:
if observables == "pos-vel":
    svd_data = 'oDataTrunc'
elif observables == "delay-embedding":
    svd_data = 'yDataTrunc'

Xsnapshots = np.hstack([yDataTrunc[1] for yDataTrunc in Data[svd_data]])
Xsnapshots = Xsnapshots[:, :]

show_modes = 9
Xsnapshots = np.hstack([DataTrunc[1] for DataTrunc in Data[svd_data]])
v, s = utils.sparse_svd(Xsnapshots, up_to_mode=max(SSMDim, show_modes))

In [ ]:
# Plot variance description: we expect three modes to capture almost all variance.
# Note we assume data centered around the origin, which is the fixed point of our system.
plt.close('all')
plot.pca_modes(s**2, up_to_mode=show_modes)

### Project data onto dominant modes

In [ ]:
Data['etaDataTrunc'] = deepcopy(Data['oDataTrunc'])
if observables == "delay-embedding":
    Vde = v[:, :SSMDim]
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ Data[svd_data][i][1]
elif observables == "pos-vel":
    assert SSMDim % 2 == 0
    Vde = np.kron(np.eye(2), v[:, :SSMDim//2])
    for i in range(nTRAJ):
        Data['etaDataTrunc'][i][1] = Vde.T @ np.vstack((Data[svd_data][i][1], np.gradient(Data[svd_data][i][1], DT, axis=1)))

In [ ]:
plt.close('all')
# plot first three reduced coordinates
plot.traj_xyz(Data,
              xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
              xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'])

### Train and test data (train/test split)

In [ ]:
indTest = [0, 8, 16, 24]
indTrain = [i for i in range(nTRAJ) if i not in indTest]

## SSMLearnPy

In [ ]:
from ssmlearnpy import SSMLearn

Use SSMLearnPy package to find parametrization of SSM (graph style) and reduced dynamics on the SSM.

In [ ]:
ssm = SSMLearn(
    t=[Data['oDataTrunc'][i][0] for i in indTrain], 
    x=[Data['yDataTrunc'][i][1] for i in indTrain], 
    reduced_coordinates=[Data['etaDataTrunc'][i][1] for i in indTrain],
    ssm_dim=SSMDim, 
    dynamics_type=RDType
)

In [ ]:
ssm.get_parametrization(poly_degree=SSMOrder)    
ssm.get_reduced_dynamics(poly_degree=ROMOrder)

Save the relevant coefficents and parameters into dictionairies which resemble the outputs of the Matlab SSMLearn package.

In [ ]:
IMInfo = {'parametrization': {}, 'chart': {}}
IMInfo['parametrization']['polynomialOrder'] = SSMOrder
IMInfo['parametrization']['H'] = ssm.decoder.map_info['coefficients']
RDInfo = {'reducedDynamics': {}}
RDInfo['reducedDynamics']['polynomialOrder'] = ROMOrder
RDInfo['reducedDynamics']['coefficients'] = ssm.reduced_dynamics.map_info['coefficients']
RDInfo['reducedDynamics']['eigenvaluesLinPartFlow'] = ssm.reduced_dynamics.map_info['eigenvalues_lin_part']
RDInfo['dynamicsType'] = RDType

## SSMLearn (Matlab)

Start Matlab engine and install/run SSMLearn

In [ ]:
# ssm = utils.start_matlab_ssmlearn("/home/jonas/Projects/stanford/SSMR-for-control")

Bring data in a format that is accepted by the (slightly modified) version of SSMLearn (requires a specific cell array structure for input data)

In [ ]:
# yDataTruncTrain_matlab = utils.numpy_to_matlab([Data['yDataTrunc'][i] for i in indTrain])
# etaDataTruncTrain_matlab = utils.numpy_to_matlab([Data['etaDataTrunc'][i] for i in indTrain])

### Learn geometry of the SSM

Find parametrization of SSM using SSMLearn

In [ ]:
# IMInfo = ssm.IMGeometry(yDataTruncTrain_matlab, SSMDim, SSMOrder,
#                         'reducedCoordinates', etaDataTruncTrain_matlab, 'l_vals', 0.)
# if observables == "pos-vel":
#     IMInfoInv = ssm.IMGeometry(etaDataTruncTrain_matlab, obsDim, SSMOrder,
#                             'reducedCoordinates', yDataTruncTrain_matlab)
#     for key in ['map', 'polynomialOrder', 'dimension', 'nonlinearCoefficients', 'phi', 'exponents', 'H']:
#         IMInfo['chart'][key] = IMInfoInv['parametrization'][key]

### Learn dynamics on the SSM (reduced dynamics)

Find parametrization of reduced dynamics using SSMLearn

In [ ]:
# RDInfo = ssm.IMDynamicsFlow(etaDataTruncTrain_matlab, 'R_PolyOrd', ROMOrder, 'style', 'default', 'l_vals', 0.)

Stop Matlab engine -- not needed anymore

In [ ]:
# ssm.quit()

Convert all matlab double arrays to numpy arrays

In [ ]:
# utils.matlab_info_dict_to_numpy(IMInfo)
# utils.matlab_info_dict_to_numpy(RDInfo)

Stability analysis of reduced dynamics

In [ ]:
# # assert np.all(np.real(RDInfo['eigenvaluesLinPartFlow']) < 0)
# RDInfo['eigenvaluesLinPartFlow']

## Analyze the obtained mappings of SSM geometry and reduced dynamics

### SSM geometry (parametrization)

In [ ]:
trajRec = {}
# geometry error
meanErrorGeo = {}
trajRec['geo'] = utils.lift_trajectories(IMInfo, Data['etaDataTrunc'])
normedTrajDist = utils.compute_trajectory_errors(trajRec['geo'], Data['yDataTrunc'])[0] * 100
meanErrorGeo['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorGeo['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average parametrization train error: {meanErrorGeo['Train']:.4e}")
print(f"Average parametrization test error: {meanErrorGeo['Test']:.4e}")

In [ ]:
# plot comparison of SSM-predicted vs. actual test trajectories
plt.close('all')
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('geo', 0), ('geo', 1), ('geo', 2)],
            xyz_names=[r'$x$ [mm]', r'$y$ [mm]', r'$z$ [mm]'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

### Reduced dynamics

In [ ]:
# reduced dynamics error
meanErrorDyn = {}
trajRec['rd'] = utils.advectRD(RDInfo, Data['etaDataTrunc'])[0]
normedTrajDist = utils.compute_trajectory_errors(trajRec['rd'], Data['etaDataTrunc'])[0] * 100
meanErrorDyn['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorDyn['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average dynamics train error: {meanErrorDyn['Train']:.4f}")
print(f"Average dynamics test error: {meanErrorDyn['Test']:.4f}")

In [ ]:
axs = plot.traj_xyz(Data,
                    xyz_idx=[('etaDataTrunc', 0), ('etaDataTrunc', 1), ('etaDataTrunc', 2)],
                    xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('rd', 0), ('rd', 1), ('rd', 2)],
            xyz_names=[r'$x_1$', r'$x_2$', r'$x_3$'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

### Global error

In [ ]:
# global error
meanErrorGlo = {}
trajRec['glob'] = utils.lift_trajectories(IMInfo, trajRec['rd'])
normedTrajDist = utils.compute_trajectory_errors(trajRec['glob'], Data['yDataTrunc'])[0] * 100
meanErrorGlo['Train'] = np.mean(normedTrajDist[indTrain])
meanErrorGlo['Test'] = np.mean(normedTrajDist[indTest])
print(f"Average global train error: {meanErrorGlo['Train']:.4f}")
print(f"Average global test error: {meanErrorGlo['Test']:.4f}")

In [ ]:
axs = plot.traj_xyz(Data,
                    xyz_idx=[('yData', 0), ('yData', 1), ('yData', 2)],
                    xyz_names=[r'$x$', r'$y$', r'$z$'],
                    traj_idx=indTest,
                    show=False)
plot.traj_xyz(trajRec,
            xyz_idx=[('glob', 0), ('glob', 1), ('glob', 2)],
            xyz_names=[r'$x$', r'$y$', r'$z$'],
            traj_idx=indTest,
            axs=axs, ls=':', color='darkblue')

## Understand which mode is responsible for what motion

In [ ]:
if observables == "pos-vel":
    modesDir, modesFreq = utils.dominant_displacement_modes(RDInfo, Vde, SSMDim=SSMDim, tip_node=TIP_NODE, n_nodes=N_NODES)
    plot.mode_direction(modesDir, modesFreq)

## Control

### Setup model for control

In [ ]:
Rauton = lambda x: RDInfo['reducedDynamics']['coefficients'] @ utils.phi(x, RDInfo['reducedDynamics']['polynomialOrder'])
Vauton = lambda x: IMInfo['parametrization']['H'] @ utils.phi(x, IMInfo['parametrization']['polynomialOrder'])
if observables == "pos-vel":
    Wauton = lambda y: IMInfo['chart']['H'] @ utils.phi(y, IMInfo['chart']['polynomialOrder'])
elif observables == "delay-embedding":
    Wauton = lambda y: Vde.T @ y

### Learn control matrix $B$

Use randomly sampled inputs as the training and test data

In [ ]:
inputData_dir = join(data_dir, "open-loop")
(t, z), u = utils.import_pos_data(inputData_dir, rest_file, TIP_NODE, return_inputs=True)
z = (z.T - q_eq).T
u = (u.T - u_eq).T
print(q_eq)

Compute observables and reduced coordinates

In [ ]:
y = assemble_observables(z)
x = Wauton(y)

Train/test split

In [ ]:
train_ratio = 0.8
split_idx = int(train_ratio * len(t))
t_train, t_test = t[:split_idx], t[split_idx:]
z_train, z_test = z[:, :split_idx], z[:, split_idx:]
y_train, y_test = y[:, :split_idx], y[:, split_idx:]
u_train, u_test = u[:, :split_idx], u[:, split_idx:]
x_train, x_test = x[:, :split_idx], x[:, split_idx:]

Plot the training data

In [ ]:
plt.close('all')
plot.traj_xyz_txyz(t=t_train,
                   x=z_train[0, :], y=z_train[1, :], z=z_train[2, :])
plot.inputs(t_train, u_train)

Plot the test data

In [ ]:
plt.close('all')
plot.traj_xyz_txyz(t=t_test,
                   x=z_test[0, :], y=z_test[1, :], z=z_test[2, :])
plot.inputs(t_test, u_test)

Compute gradient as predicted by reduced dynamics as well as the numerical (true) gradient

In [ ]:
dxdt = np.gradient(x_train, DT, axis=1)
dxdt_ROM = Rauton(x_train)

Plot gradients (numerical differentiation and reduced dynamics)

In [ ]:
plot.reduced_coordinates_gradient(t_train, [dxdt, dxdt_ROM], labels=["true numerical", "predicted autonomous"], how="all")

In [ ]:
# plt.close('all')
# plot.dependence_of_xdot_on_inputs(dxDt, u_train_trunc)

Learn $B$ matrix

In [ ]:
from sklearn.linear_model import Ridge

polyUorder = 1
# assemble_features = lambda u, x: utils.phi(np.vstack([u, x]), order=polyUorder)
assemble_features = lambda u, x: utils.phi(u, order=polyUorder)

X = assemble_features(u_train, x_train)

B_learn = utils.regress_B(X, dxdt, dxdt_ROM, alpha=0, method='ridge')
print(f"Frobenius norm of B_learn: {np.linalg.norm(B_learn, ord='fro'):.4f}")

Plot gradients again, but this time, using the learned influence of inputs for the prediction

In [ ]:
plt.close('all')
plot_reduced_coords = np.s_[:] # [3, 4, 5]
dxDt_ROM_with_B = Rauton(x_train) + B_learn @ assemble_features(u_train, x_train)
plot.reduced_coordinates_gradient(t_train, [dxdt[plot_reduced_coords, :], dxdt_ROM[plot_reduced_coords, :], dxDt_ROM_with_B[plot_reduced_coords, :]], labels=["true numerical", "predicted autonomous", "predicted with inputs"], how="all")

In [ ]:
# # home-brewed ridge regression
# alpha = 0
# X, y = U.T, (dXbarDt - dXbarDt_ROM).T
# B_learn = np.linalg.solve(X.T @ X + alpha * np.eye(X.shape[0]), X.T @ y)

### Integrate model with inputs using learned influence of control (open-loop prediction)

Try to predict the different test trajectories using _non-autonomous_ reduced dynamics

In [ ]:
R = lambda x, u: Rauton(np.atleast_2d(x)) + B_learn @ assemble_features(u, x)

Import test trajectory data

In [ ]:
test_results = {}
test_trajectory_folders = ['open-loop_circle']
test_trajectories = [{
        'name': "like training data",
        't': t_test,
        'z': z_test,
        'u': u_test,
        'x': x_test
    }]
for test_traj in test_trajectory_folders:
    traj_dir = join(robot_dir, "dataCollection", test_traj)
    (t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                      rest_file=rest_file,
                                      output_node=TIP_NODE, return_inputs=True, traj_index=0)
    z = (z.T - q_eq).T
    u = (u.T - u_eq).T
    y = assemble_observables(z)
    x = Wauton(y)
    test_trajectories.append({
            'name': test_traj,
            't': t,
            'z': z,
            'u': u,
            'x': x
        })

Do open loop prediction for all test trajectories; plot and save results

In [ ]:
for traj in test_trajectories:
    z_pred = utils.predict_open_loop(R, Vauton, traj['t'], traj['u'], x0=traj['x'][:, 0])
    rmse = float(np.sum(np.sqrt(np.mean((z_pred[:3, :] - traj['z'][:3])**2, axis=0))) / len(traj['t']))
    print(f"({traj['name']}): RMSE = {rmse:.4f}")
    test_results[traj['name']] = {
        'RMSE': rmse
    }
    axs = plot.traj_xyz_txyz(traj['t'],
                                z_pred[0, :], z_pred[1, :], z_pred[2, :],
                                show=False)
    axs = plot.traj_xyz_txyz(traj['t'],
                                traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
                                color="tab:orange", axs=axs, show=False)
    axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
    plt.show()
print(f"(overall): RMSE = {np.mean([test_results[traj]['RMSE'] for traj in test_results]):.4f}")

## Save SSM model

In [ ]:
# settings = {
#     'SSMDim': SSMDim,
#     'SSMOrder': SSMOrder,
#     'ROMOrder': ROMOrder,
#     'observables': observables,
#     'poly_u_order': polyUorder,
#     'oDOF': oDOF,
#     'input_dim': INPUT_DIM,
#     'n_delay': N_DELAY
# }
# utils.save_ssm_model(join(robot_dir, "SSMmodels"), RDInfo, IMInfo, B_learn, Vde, settings, test_results)